In [2]:
import pandas as pd

df = pd.read_csv("archive2/captions.txt")
processed_dict = {}
caption_list = []

for index, row in df.iterrows():
    if row['image'] not in processed_dict:
        processed_dict[row['image']] = []
    processed_dict[row['image']].append(row['caption'])
    caption_list.append(row['caption'])

print(processed_dict['1001773457_577c3a7d70.jpg'])

['A black dog and a spotted dog are fighting', 'A black dog and a tri-colored dog playing with each other on the road .', 'A black dog and a white dog with brown spots are staring at each other in the street .', 'Two dogs of different breeds looking at each other on the road .', 'Two dogs on pavement moving toward each other .']


In [3]:
type(caption_list[0])

str

In [4]:
print(len(caption_list))

40455


In [6]:
from PIL import Image
import torch

from transformers import CLIPProcessor, CLIPModel,AutoTokenizer

device = "cuda" if torch.cuda.is_available() else \
         ("mps" if torch.backends.mps.is_available() else "cpu" ) 
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

image_path = 'archive2/Images/1000268201_693b08cb0e.jpg'
image = Image.open(image_path)
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
tokenized_captions = tokenizer(caption_list, padding=True)
inputs = processor(text=caption_list, images=image, return_tensors="pt", padding=True).to(device)

outputs = model(**inputs)

RuntimeError: MPS backend out of memory (MPS allocated: 78.75 GB, other allocations: 1.23 GB, max allowed: 81.60 GB). Tried to allocate 3.24 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [11]:
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score

In [12]:
print(logits_per_image)

tensor([[19.1131, 18.8582]], grad_fn=<TBackward0>)


In [13]:
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [14]:

print(probs)

tensor([[0.5634, 0.4366]], grad_fn=<SoftmaxBackward0>)
